Let's finish off our investigation of genetic algorithms by revisiting our first problem, [Sum-To-N](./sum_to_n.ipynb), with a new twist &hellip;

## __Sum to M & N__

**Problem:**

This time, we will have 2 arrays:

- `arr_1` and `arr_2`

- each array will contain `r` numbers $\in \{1..50\}$

For each index $i$, we may take the number from `arr_1` or `arr_2`,  
but _not both_.

We want to find the best pair of subsets (of each array) such that:

- the sum of elements taken from `arr_1` is closest to `M` without going over

- the sum of elements taken from `arr_2` is closest to `N` without going over

This problem is more complex as it requires 2 optimizations to be done at once, and has a restraint on selecting elements.

Let's solve this problem with ease using our powerful [__pyvolver__](./pyvolver/) package!

In [1]:
import pyvolver
import random

# Problem setup
r = 20 # length of arrays
ARR_1 = tuple(random.randint(1,50) for _ in range(r))
ARR_2 = tuple(random.randint(1,50) for _ in range(r))

M = random.randint(0, 50 * r)
N = random.randint(0, 50 * r)

# show us the selection
print(f'ARR_1: {ARR_1}')
print(f'ARR_2: {ARR_2}')
print()
print(f'M: {M}\nN: {N}')

ARR_1: (28, 50, 37, 23, 14, 43, 34, 14, 47, 4, 28, 34, 3, 20, 50, 33, 31, 30, 24, 22)
ARR_2: (44, 1, 35, 34, 10, 34, 40, 4, 34, 40, 38, 45, 35, 2, 12, 15, 33, 18, 21, 26)

M: 717
N: 649


In [2]:
# We will create 2 chromosomes
# -- each will represent a possible subset for one of the arrays
# -- each will use single-point crossover
chrom_1 = pyvolver.create_symbolic_chromosome(r, '01', _name='Arr1')
chrom_2 = pyvolver.create_symbolic_chromosome(r, '01', _name='Arr2')

# Next, we make a species to hold these chromosomes, and evaluate its fitness
def to_solution(self: pyvolver.Organism):
	# map binary strings to subsets
	size = self.genome[0].length
	subset_1 = [ARR_1[i] for i in range(size) if self.genome[0][i] == '1']
	subset_2 = [ARR_2[i] for i in range(size) if self.genome[1][i] == '1']
	return (subset_1, subset_2)

def get_fitness(self: pyvolver.Organism):
	length = self.genome[0].length

	# sum each subset
	subsets = self.to_solution()
	sum_1 = sum(subsets[0]) # chromosome for arr_1
	sum_2 = sum(subsets[1]) # chromosome for arr_2

	''' Invalid Solutions (penalize fitness):
		1) sum_1 exceeds M
		2) sum_2 exceeds N
		3) the ith element of arr1 AND arr2 were both chosen
	'''
	penalty = 0
	if sum_1 > M:
		penalty += 10 * length
	if sum_2 > N:
		penalty += 10 * length
	for i in range(length):
		if self.genome[0][i] == '1' == self.genome[1][i]:
			penalty += 10
	if penalty > 0:
		return -penalty
	
	# the closer the better, for each subset
	return sum_1 + sum_2

species = pyvolver.create_species('Sum_M_N', get_fitness, [chrom_1, chrom_2], to_solution=to_solution)

# Finally, we start a society of initial solutions
pop = pyvolver.Population(species)

In [7]:
pop.evolve(100)
print(pop)

--- Generation 500 ---

Member 1)
  Arr1:           1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0 0 0
  Arr2:           0 0 1 1 0 0 1 1 0 0 1 0 1 1 1 0 0 1 1 1
  Fitness:        492

Member 2)
  Arr1:           1 1 0 0 0 0 1 0 1 1 0 1 1 0 0 1 1 0 0 0
  Arr2:           0 0 1 1 0 0 1 1 0 0 1 0 0 1 1 0 0 1 1 1
  Fitness:        492

Member 3)
  Arr1:           1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0
  Arr2:           0 0 1 1 0 0 1 1 0 0 1 0 1 1 1 0 0 1 1 1
  Fitness:        492

Member 4)
  Arr1:           1 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 1 0 0 0
  Arr2:           0 0 1 1 0 0 1 1 0 0 1 0 1 1 1 0 0 1 1 1
  Fitness:        492

Member 5)
  Arr1:           1 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0
  Arr2:           0 0 1 1 0 0 1 1 0 0 1 0 1 1 1 0 0 1 1 1
  Fitness:        492

Member 6)
  Arr1:           1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0 0 1
  Arr2:           0 0 1 1 0 0 1 1 0 0 1 0 1 1 1 0 0 1 1 1
  Fitness:        492

Member 7)
  Arr1:           1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0 0 0
  Arr2:       

We can see from the genes displayed on the two chromosomes that the XOR restriction was achieved.

For each loci, we never have a 1 on both chromosomes.

In [14]:
# Let's decode our best answer
ans = pop.solution

subsets = ans.to_solution() # our custom method from before

for i in range(2):
	print(f'arr_{i+1}: {subsets[i]}')
	print(f'sum: {sum(subsets[i])}')
	if i == 0:
		print(f'M: {M}')
	else:
		print(f'N: {N}')
	print()

arr_1: [28, 50, 47, 4, 34, 33, 31]
sum: 227
M: 717

arr_2: [35, 34, 40, 4, 38, 35, 2, 12, 18, 21, 26]
sum: 265
N: 649



We have met the other constraint too:  
> The sum of elements in each subset does not exceed the target assigned to it.

The `pyvolver` package can help us solve _all kinds_ of combinatorial, optimization and permutational problems, with little setup required.

We have developed a powerful AI tool that can help us whenever we need it next!